### Silver to Gold: Building BI Ready Tables

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, FloatType
from pyspark.sql import Row

In [0]:
catalog_name = "ecommerce"

### Products 

In [0]:
df_products = spark.table(f"{catalog_name}.silver.slv_products")
df_brands = spark.table(f"{catalog_name}.silver.slv_brands")
df_category = spark.table(f"{catalog_name}.silver.slv_category")

In [0]:
df_products.createOrReplaceTempView("v_products")
df_brands.createOrReplaceTempView("v_brands")
df_category.createOrReplaceTempView("v_category")

In [0]:
display(spark.sql("select * from v_products limit 5"))

product_id,sku,category_code,brand_code,color,size,material,weight_grams,length_cm,width_cm,height_cm,rating_count,file_name,ingest_timestamp
2000000000015,STCR-HNK-00001,HNK,STCR,White,One-Size,Cotton,305,22.2,17.1,6.3,0,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/products/products.csv,2025-11-08T05:11:34.097Z
2000000000022,HMNS-HNK-00002,HNK,HMNS,Silver,One-Size,Steel,682,18.2,12.3,3.7,1,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/products/products.csv,2025-11-08T05:11:34.097Z
2000000000039,NOVW-CE-00003,CE,NOVW,Purple,One-Size,Wood,243,18.2,13.9,4.2,0,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/products/products.csv,2025-11-08T05:11:34.097Z
2000000000046,URTL-APP-00004,APP,URTL,Silver,S,Rubber,225,17.6,4.6,5.8,50,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/products/products.csv,2025-11-08T05:11:34.097Z
2000000000053,GGRN-GRC-00005,GRCY,GGRN,Silver,One-Size,Rubber,455,27.2,15.8,7.4,4,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/products/products.csv,2025-11-08T05:11:34.097Z


In [0]:
display(spark.sql('select * from v_category limit 5'))

category_code,category_name,_ingested_at,_source_file
BKS,Books,2025-11-08T05:11:05.835Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/category/category.csv
GRCY,Grocery,2025-11-08T05:11:05.835Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/category/category.csv
HNK,Home & Kitchen,2025-11-08T05:11:05.835Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/category/category.csv
TOY,Toys & Games,2025-11-08T05:11:05.835Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/category/category.csv
APP,Apparel,2025-11-08T05:11:05.835Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/category/category.csv


In [0]:
display(spark.sql('select * from v_brands limit 5'))

brand_code,brand_name,category_code,_source_file,ingested_at
ACME,AcmeTech,CE,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/brands/brands.csv,2025-11-08T05:10:15.848Z
NOVW,NovaWave,CE,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/brands/brands.csv,2025-11-08T05:10:15.848Z
ZNTH,Zenith,CE,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/brands/brands.csv,2025-11-08T05:10:15.848Z
BYTM,ByteMax,CE,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/brands/brands.csv,2025-11-08T05:10:15.848Z
ECOT,EcoTone,CE,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/brands/brands.csv,2025-11-08T05:10:15.848Z


In [0]:

# Make sure we’re on the right catalog
spark.sql(f"USE CATALOG {catalog_name}")

DataFrame[]

In [0]:
%sql

-- Build brands×category mapping and write Gold table
CREATE OR REPLACE TABLE gold.gld_dim_products AS

WITH brands_categories AS (
  SELECT
    b.brand_name,
    b.brand_code,
    c.category_name,
    c.category_code
  FROM v_brands b
  INNER JOIN v_category c
  ON
    b.category_code = c.category_code
)
SELECT
  p.product_id,
  p.sku,
  p.category_code,
  COALESCE(bc.category_name, 'Not Available') AS category_name,
  p.brand_code,
  COALESCE(bc.brand_name, 'Not Available')   AS brand_name,
  p.color,
  p.size,
  p.material,
  p.weight_grams,
  p.length_cm,
  p.width_cm,
  p.height_cm,
  p.rating_count,
  p.file_name,
  p.ingest_timestamp
FROM v_products p
LEFT JOIN brands_categories bc
  ON p.brand_code = bc.brand_code;

num_affected_rows,num_inserted_rows


### Customers

In [0]:
# India states
india_region = {
    "MH": "West", "GJ": "West", "RJ": "West",
    "KA": "South", "TN": "South", "TS": "South", "AP": "South", "KL": "South",
    "UP": "North", "WB": "North", "DL": "North"
}
# Australia states
australia_region = {
    "VIC": "SouthEast", "WA": "West", "NSW": "East", "QLD": "NorthEast"
}

# United Kingdom states
uk_region = {
    "ENG": "England", "WLS": "Wales", "NIR": "Northern Ireland", "SCT": "Scotland"
}

# United States states
us_region = {
    "MA": "NorthEast", "FL": "South", "NJ": "NorthEast", "CA": "West", 
    "NY": "NorthEast", "TX": "South"
}

# UAE states
uae_region = {
    "AUH": "Abu Dhabi", "DU": "Dubai", "SHJ": "Sharjah"
}

# Singapore states
singapore_region = {
    "SG": "Singapore"
}

# Canada states
canada_region = {
    "BC": "West", "AB": "West", "ON": "East", "QC": "East", "NS": "East", "IL": "Other"
}

# Combine into a master dictionary
country_state_map = {
    "India": india_region,
    "Australia": australia_region,
    "United Kingdom": uk_region,
    "United States": us_region,
    "United Arab Emirates": uae_region,
    "Singapore": singapore_region,
    "Canada": canada_region
}  


In [0]:
country_state_map

{'India': {'MH': 'West',
  'GJ': 'West',
  'RJ': 'West',
  'KA': 'South',
  'TN': 'South',
  'TS': 'South',
  'AP': 'South',
  'KL': 'South',
  'UP': 'North',
  'WB': 'North',
  'DL': 'North'},
 'Australia': {'VIC': 'SouthEast',
  'WA': 'West',
  'NSW': 'East',
  'QLD': 'NorthEast'},
 'United Kingdom': {'ENG': 'England',
  'WLS': 'Wales',
  'NIR': 'Northern Ireland',
  'SCT': 'Scotland'},
 'United States': {'MA': 'NorthEast',
  'FL': 'South',
  'NJ': 'NorthEast',
  'CA': 'West',
  'NY': 'NorthEast',
  'TX': 'South'},
 'United Arab Emirates': {'AUH': 'Abu Dhabi', 'DU': 'Dubai', 'SHJ': 'Sharjah'},
 'Singapore': {'SG': 'Singapore'},
 'Canada': {'BC': 'West',
  'AB': 'West',
  'ON': 'East',
  'QC': 'East',
  'NS': 'East',
  'IL': 'Other'}}

In [0]:
# 1 Flatten country_state_map into a list of Rows
rows = []
for country, states in country_state_map.items():
    for state_code, region in states.items():
        rows.append(Row(country=country, state=state_code, region=region))
rows[:10]        

[Row(country='India', state='MH', region='West'),
 Row(country='India', state='GJ', region='West'),
 Row(country='India', state='RJ', region='West'),
 Row(country='India', state='KA', region='South'),
 Row(country='India', state='TN', region='South'),
 Row(country='India', state='TS', region='South'),
 Row(country='India', state='AP', region='South'),
 Row(country='India', state='KL', region='South'),
 Row(country='India', state='UP', region='North'),
 Row(country='India', state='WB', region='North')]

In [0]:
# 2️ Create mapping DataFrame
df_region_mapping = spark.createDataFrame(rows)

# Optional: show mapping
df_region_mapping.show(truncate=False)

+--------------+-----+----------------+
|country       |state|region          |
+--------------+-----+----------------+
|India         |MH   |West            |
|India         |GJ   |West            |
|India         |RJ   |West            |
|India         |KA   |South           |
|India         |TN   |South           |
|India         |TS   |South           |
|India         |AP   |South           |
|India         |KL   |South           |
|India         |UP   |North           |
|India         |WB   |North           |
|India         |DL   |North           |
|Australia     |VIC  |SouthEast       |
|Australia     |WA   |West            |
|Australia     |NSW  |East            |
|Australia     |QLD  |NorthEast       |
|United Kingdom|ENG  |England         |
|United Kingdom|WLS  |Wales           |
|United Kingdom|NIR  |Northern Ireland|
|United Kingdom|SCT  |Scotland        |
|United States |MA   |NorthEast       |
+--------------+-----+----------------+
only showing top 20 rows


In [0]:
df_silver = spark.table(f'{catalog_name}.silver.slv_customers')
display(df_silver.limit(5))

customer_id,phone,country_code,country,state,file_name,ingest_timestamp
CUST000000000001,917280033536.0,IN,India,MH,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z
CUST000000000002,619489725433.0,AU,Australia,VIC,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z
CUST000000000003,919390066524.0,IN,India,TN,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z
CUST000000000004,917073741793.0,IN,India,TN,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z
CUST000000000005,618478772532.0,AU,Australia,WA,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z


In [0]:
df_gold = df_silver.join(df_region_mapping, on=['country', 'state'], how='left')

df_gold = df_gold.fillna({'region': 'Other'})

display(df_gold.limit(5))

country,state,customer_id,phone,country_code,file_name,ingest_timestamp,region
India,MH,CUST000000000001,917280033536.0,IN,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z,West
Australia,VIC,CUST000000000002,619489725433.0,AU,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z,SouthEast
India,TN,CUST000000000003,919390066524.0,IN,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z,South
India,TN,CUST000000000004,917073741793.0,IN,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z,South
Australia,WA,CUST000000000005,618478772532.0,AU,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/customers/customers.csv,2025-11-08T05:11:58.005Z,West


In [0]:
# Write raw data to the gold layer (catalog: ecommerce, schema: gold, table: gld_dim_customers)
df_gold.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.gold.gld_dim_customers")

### Date/Calendar

In [0]:
df_silver = spark.table(f'{catalog_name}.silver.slv_calendar')
display(df_silver.limit(5))

date,year,day_name,quarter,week,_ingested_at,_source_file
2025-09-07,2025,Sunday,Q3-2025,Week36-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
2025-08-06,2025,Wednesday,Q3-2025,Week32-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
2025-09-23,2025,Tuesday,Q3-2025,Week39-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
2025-08-25,2025,Monday,Q3-2025,Week35-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
2025-10-16,2025,Thursday,Q4-2025,Week42-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv


In [0]:
df_gold = df_silver.withColumn("date_id", F.date_format(F.col("date"), "yyyyMMdd").cast("int"))

# Add month name (e.g., 'January', 'February', etc.)
df_gold = df_gold.withColumn("month_name", F.date_format(F.col("date"), "MMMM"))

# Add is_weekend column
df_gold = df_gold.withColumn(
    "is_weekend",
    F.when(F.col("day_name").isin("Saturday", "Sunday"), 1).otherwise(0)
)

display(df_gold.limit(5))


date,year,day_name,quarter,week,_ingested_at,_source_file,date_id,month_name,is_weekend
2025-09-07,2025,Sunday,Q3-2025,Week36-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv,20250907,September,1
2025-08-06,2025,Wednesday,Q3-2025,Week32-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv,20250806,August,0
2025-09-23,2025,Tuesday,Q3-2025,Week39-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv,20250923,September,0
2025-08-25,2025,Monday,Q3-2025,Week35-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv,20250825,August,0
2025-10-16,2025,Thursday,Q4-2025,Week42-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv,20251016,October,0


In [0]:
desired_columns_order = ["date_id", "date", "year", "month_name", "day_name", "is_weekend", "quarter", "week", "_ingested_at", "_source_file"]

df_gold = df_gold.select(desired_columns_order)

display(df_gold.limit(5))

date_id,date,year,month_name,day_name,is_weekend,quarter,week,_ingested_at,_source_file
20250907,2025-09-07,2025,September,Sunday,1,Q3-2025,Week36-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
20250806,2025-08-06,2025,August,Wednesday,0,Q3-2025,Week32-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
20250923,2025-09-23,2025,September,Tuesday,0,Q3-2025,Week39-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
20250825,2025-08-25,2025,August,Monday,0,Q3-2025,Week35-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv
20251016,2025-10-16,2025,October,Thursday,0,Q4-2025,Week42-2025,2025-11-08T05:12:21.542Z,dbfs:/Volumes/ecommerce/source_data/ecommerce_store/date/date.csv


In [0]:
# write table to gold layer
df_gold.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.gold.gld_dim_date")

In [0]:
%sql

DESCRIBE EXTENDED ecommerce.gold.gld_dim_date;

col_name,data_type,comment
date_id,int,null
date,date,null
year,int,null
month_name,string,null
day_name,string,null
is_weekend,int,null
quarter,string,null
week,string,null
_ingested_at,timestamp,null
_source_file,string,null
